# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""404173051456653878""","""../../../../data/SampleHDS.jso…",4.9159e18,"""4692280470527271555""","""10302775680610657704""","""3883180112191679735""",false
"""16770761098689782295""","""../../../../data/SampleHDS.jso…",3.9247e18,"""5953402813748492241""","""10302775680610657704""","""1937549559038232991""",true
"""16924908019897135234""","""../../../../data/SampleHDS.jso…",1.5734e19,"""12652296150669527045""","""17353787005530307570""","""3883180112191679735""",false
"""16770761098689782295""","""../../../../data/SampleHDS.jso…",9.6042e18,"""13830154787914866761""","""12179607937839748158""","""3883180112191679735""",false
"""404173051456653878""","""../../../../data/SampleHDS.jso…",1.7984e19,"""9183950266408627032""","""12179607937839748158""","""4354866275269377784""",true
"""16924908019897135234""","""../../../../data/SampleHDS.jso…",7.6710e18,"""8471678075449575800""","""2914612472534327262""",null,false
"""16924908019897135234""","""../../../../data/SampleHDS.jso…",3.3658e18,"""15737694272008311484""","""12179607937839748158""","""4354866275269377784""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_MaritalStatus': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_2',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""12759996022709740427""","""../../../../data/SampleHDS.jso…",1.6045e19,"""3420116485914154392""","""FirstMortgage30yr""","""16808286123624275033""","""Rejected"""
"""12277049264649131524""","""../../../../data/SampleHDS.jso…",1.1771e19,"""12076802019602216286""","""FirstMortgage30yr""","""18200265145214301830""","""Accepted"""
"""4057306347847713165""","""../../../../data/SampleHDS.jso…",5.5013e18,"""3212651797849668061""","""MoneyMarketSavingsAccount""","""16808286123624275033""","""Rejected"""
"""12277049264649131524""","""../../../../data/SampleHDS.jso…",1.3401e18,"""1292810103145664408""","""BasicChecking""","""16808286123624275033""","""Rejected"""
"""12759996022709740427""","""../../../../data/SampleHDS.jso…",8.8584e18,"""9675600202102410476""","""BasicChecking""","""17572559607884776069""","""Accepted"""
"""4057306347847713165""","""../../../../data/SampleHDS.jso…",2.0311e18,"""8567295650841513926""","""UPlusFinPersonal""",null,"""Rejected"""
"""4057306347847713165""","""../../../../data/SampleHDS.jso…",1.2661e19,"""14860297069384159990""","""BasicChecking""","""17572559607884776069""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""15704902589394036773""","""../../../../data/SampleHDS.jso…",1.2459e19,"""5395477463980974623""","""FirstMortgage30yr""","""8581908620437591436""","""Rejected"""
"""7993606177582160336""","""../../../../data/SampleHDS.jso…",4.7993e18,"""4439544387414767747""","""FirstMortgage30yr""","""3124974601909306092""","""Accepted"""
"""4922010897597875305""","""../../../../data/SampleHDS.jso…",1.3076e19,"""4712562603249850485""","""MoneyMarketSavingsAccount""","""8581908620437591436""","""Rejected"""
"""7993606177582160336""","""../../../../data/SampleHDS.jso…",1.6706e19,"""18171885522261281229""","""BasicChecking""","""8581908620437591436""","""Rejected"""
"""15704902589394036773""","""../../../../data/SampleHDS.jso…",1.8349e18,"""11372330336759034905""","""BasicChecking""","""2468568473826695821""","""Accepted"""
"""4922010897597875305""","""../../../../data/SampleHDS.jso…",1.2687e19,"""18344768351478480948""","""UPlusFinPersonal""",null,"""Rejected"""
"""4922010897597875305""","""../../../../data/SampleHDS.jso…",1.7229e19,"""16279688712674263256""","""BasicChecking""","""2468568473826695821""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_MaritalStatus=PREDICTOR_0
filename=filename
Customer_CLV=PREDICTOR_2
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""4680616243636161801""",1.1939e19,"""6975737182130307024""","""FirstMortgage30yr""","""17040774647327699853""","""Rejected"""
"""2813416363470635376""",4.4418e18,"""16054467463533024279""","""FirstMortgage30yr""","""14860844409794217593""","""Accepted"""
"""4882900982119994914""",1.1450e19,"""2679109607038234748""","""MoneyMarketSavingsAccount""","""17040774647327699853""","""Rejected"""
"""2813416363470635376""",6.5167e18,"""10408106888189457234""","""BasicChecking""","""17040774647327699853""","""Rejected"""
"""4680616243636161801""",1.6807e19,"""16659331222881826711""","""BasicChecking""","""11928824298899560061""","""Accepted"""
"""4882900982119994914""",9.7154e18,"""16604888074900216847""","""UPlusFinPersonal""",null,"""Rejected"""
"""4882900982119994914""",7.6805e18,"""14472806737725633635""","""BasicChecking""","""11928824298899560061""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'